In [ ]:
import numpy as np
import os
import random
from keras import backend as K
from keras.models import Model
from keras.layers.core import Activation
from keras.layers import Input, Embedding, Flatten, dot
from keras.optimizers import Adam
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_files

In [ ]:
import base64
import requests

master_input = "https://raw.githubusercontent.com/shagunsodhani/CNN-Sentence-Classifier/master/sample_dataset/input.txt"
req_input = requests.get(master_input)
req_input = req_input.text
input_list = req_input.splitlines()


master_label = "https://raw.githubusercontent.com/shagunsodhani/CNN-Sentence-Classifier/master/sample_dataset/label.txt"
req_label = requests.get(master_label)
req_label = req_label.text
label_list = req_label.splitlines()



In [ ]:
pos = []
neg = []

for i in range(len(label_list)):
  if(label_list[i]== 'positive'):
    pos.append(input_list[i])
  else:
    neg.append(input_list[i]) 

len(pos),len(neg)

(5331, 5331)

In [ ]:
random.shuffle(pos)
random.shuffle(neg)


middle = 8530 / 2

train_pos = []
test_pos = []
train_neg = []
test_neg = []

for j in range(len(pos)):
  if j < middle:
    train_pos.append(pos[j])
    train_neg.append(neg[j])
  else:
    test_pos.append(pos[j])
    test_neg.append(neg[j])

len(train_pos),len(test_pos),len(train_neg),len(test_neg)

(4265, 1066, 4265, 1066)

In [ ]:
# Create a directory called "data" if it doesn't exist already
if not os.path.exists("data"):
    os.makedirs("data")
  
if not os.path.exists("data/train"):
    os.makedirs("data/train")

if not os.path.exists("data/train/neg"):
    os.makedirs("data/train/neg")

if not os.path.exists("data/train/pos"):
    os.makedirs("data/train/pos")

if not os.path.exists("data/test"):
    os.makedirs("data/test")

if not os.path.exists("data/test/neg"):
    os.makedirs("data/test/neg")

if not os.path.exists("data/test/pos"):
    os.makedirs("data/test/pos")

In [ ]:
name = 0

for string in train_pos:
  filename = str(name) + ".txt"
  name += 1
  with open("data/train/pos/%s" %(filename), "w") as f:
        f.write(string)
        
for string in train_neg:
  filename = str(name) + ".txt"
  name += 1
  with open("data/train/neg/%s" %(filename), "w") as f:
        f.write(string)

for string in test_pos:
  filename = str(name) + ".txt"
  name += 1
  with open("data/test/pos/%s" %(filename), "w") as f:
        f.write(string)

for string in test_neg:
  filename = str(name) + ".txt"
  name += 1
  with open("data/test/neg/%s" %(filename), "w") as f:
        f.write(string)



In [ ]:
PATH_TO_IMDB = r'data'
def load_imdb_data(datadir):
    # read in training and test corpora
    categories= ['pos', 'neg']
    train_b = load_files(datadir+'/train', shuffle=True, 
                         categories=categories)
    test_b = load_files(datadir+'/test', shuffle=True,
                         categories=categories)
    train_b.data = [x.decode('utf-8') for x in train_b.data]
    test_b.data =  [x.decode('utf-8') for x in test_b.data]
    veczr =  CountVectorizer(ngram_range=(1,3), binary=True, 
                             token_pattern=r'\w+',
                             max_features=800000)
    dtm_train = veczr.fit_transform(train_b.data)
    dtm_test = veczr.transform(test_b.data)
    y_train = train_b.target
    y_test = test_b.target
    print("DTM shape (training): (%s, %s)" % (dtm_train.shape))
    print("DTM shape (test): (%s, %s)" % (dtm_train.shape))
    num_words = len([v for k,v in veczr.vocabulary_.items()]) + 1
    print('vocab size:%s' % (num_words))
  
    return (dtm_train, dtm_test), (y_train, y_test), num_words

(dtm_train, dtm_test), (y_train, y_test), num_words = load_imdb_data(PATH_TO_IMDB)

DTM shape (training): (8530, 235076)
DTM shape (test): (8530, 235076)
vocab size:235077


In [ ]:
def dtm2wid(dtm, maxlen):
    x = []
    nwds = []
    for idx, row in enumerate(dtm):
        seq = []
        indices = (row.indices + 1).astype(np.int64)
        np.append(nwds, len(indices))
        data = (row.data).astype(np.int64)
        count_dict = dict(zip(indices, data))
        for k,v in count_dict.items():
            seq.extend([k]*v)
        num_words = len(seq)
        nwds.append(num_words)
        # pad up to maxlen with 0
        if num_words < maxlen: 
            seq = np.pad(seq, (maxlen - num_words, 0),    
                         mode='constant')
        # truncate down to maxlen
        else:                  
            seq = seq[-maxlen:]
        x.append(seq)
    nwds = np.array(nwds)
    print('sequence stats: avg:%s, max:%s, min:%s' % (nwds.mean(),
                                                      nwds.max(), 
                                                      nwds.min()) )
    return np.array(x)
maxlen = 500
x_train = dtm2wid(dtm_train, maxlen)
x_test = dtm2wid(dtm_test, maxlen)

sequence stats: avg:53.29601406799531, max:144, min:1
sequence stats: avg:29.77016885553471, max:86, min:1


In [ ]:
def pr(dtm, y, y_i):
    p = dtm[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

nbratios = np.log(pr(dtm_train, y_train, 1)/pr(dtm_train, 
                                               y_train, 0))
nbratios = np.squeeze(np.asarray(nbratios))

In [ ]:
def get_model(num_words, maxlen, nbratios=None):
    # setup the embedding matrix for NB log-count ratios
    embedding_matrix = np.zeros((num_words, 1))
    for i in range(1, num_words): # skip 0, the padding value
        if nbratios is not None:
            # if log-count ratios are supplied, then it's NBSVM
            embedding_matrix[i] = nbratios[i-1]
        else:
            # if log-count ratios are not supplied, 
            # this reduces to a logistic regression
            embedding_matrix[i] = 1
    # setup the model
    inp = Input(shape=(maxlen,))
    r = Embedding(num_words, 1, input_length=maxlen, 
                  weights=[embedding_matrix], 
                  trainable=False)(inp)
    x = Embedding(num_words, 1, input_length=maxlen, 
                  embeddings_initializer='glorot_normal')(inp)
    x = dot([r,x], axes=1)
    x = Flatten()(x)
    x = Activation('sigmoid')(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])
    return model

In [ ]:
model = get_model(num_words, maxlen, nbratios=nbratios)
model.fit(x_train, y_train,
          batch_size=50,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
171/171 [==============================] - 2s 9ms/step - loss: 0.6528 - accuracy: 0.8067 - val_loss: 0.6372 - val_accuracy: 0.7753
Epoch 2/10
171/171 [==============================] - 1s 7ms/step - loss: 0.4996 - accuracy: 0.9751 - val_loss: 0.5984 - val_accuracy: 0.7777
Epoch 3/10
171/171 [==============================] - 1s 8ms/step - loss: 0.3896 - accuracy: 0.9868 - val_loss: 0.5715 - val_accuracy: 0.7791
Epoch 4/10
171/171 [==============================] - 2s 9ms/step - loss: 0.3141 - accuracy: 0.9925 - val_loss: 0.5521 - val_accuracy: 0.7795
Epoch 5/10
171/171 [==============================] - 2s 10ms/step - loss: 0.2601 - accuracy: 0.9950 - val_loss: 0.5374 - val_accuracy: 0.7781
Epoch 6/10
171/171 [==============================] - 1s 8ms/step - loss: 0.2199 - accuracy: 0.9961 - val_loss: 0.5258 - val_accuracy: 0.7786
Epoch 7/10
171/171 [==============================] - 1s 7ms/step - loss: 0.1890 - accuracy: 0.9967 - val_loss: 0.5164 - val_accuracy: 0.7800
Epoch

In [ ]:
batch_sizes = [16, 32, 64]
epochs = [5, 10, 15]

In [ ]:
for i in range(len(batch_sizes)):
  for j in range(len(epochs)):
    model = get_model(num_words, maxlen, nbratios=nbratios)
    model.fit(x_train, y_train,
          batch_size=batch_sizes[i],
          epochs=epochs[j],
          validation_data=(x_test, y_test))


Epoch 1/5
534/534 [==============================] - 5s 7ms/step - loss: 0.6266 - accuracy: 0.8208 - val_loss: 0.6051 - val_accuracy: 0.7763
Epoch 2/5
534/534 [==============================] - 3s 6ms/step - loss: 0.4133 - accuracy: 0.9746 - val_loss: 0.5587 - val_accuracy: 0.7758
Epoch 3/5
534/534 [==============================] - 4s 7ms/step - loss: 0.2875 - accuracy: 0.9900 - val_loss: 0.5317 - val_accuracy: 0.7763
Epoch 4/5
534/534 [==============================] - 3s 6ms/step - loss: 0.2130 - accuracy: 0.9947 - val_loss: 0.5138 - val_accuracy: 0.7781
Epoch 5/5
534/534 [==============================] - 3s 6ms/step - loss: 0.1644 - accuracy: 0.9965 - val_loss: 0.5007 - val_accuracy: 0.7805
Epoch 1/10
534/534 [==============================] - 4s 7ms/step - loss: 0.6265 - accuracy: 0.8161 - val_loss: 0.6048 - val_accuracy: 0.7767
Epoch 2/10
534/534 [==============================] - 4s 8ms/step - loss: 0.4135 - accuracy: 0.9743 - val_loss: 0.5586 - val_accuracy: 0.7758
Epoch 3/10
